In [ ]:
if True:
    %matplotlib notebook
elif False:
    %matplotlib inline
else:
    %matplotlib qt    

In [ ]:
import sys
import os
import glob
import getpass

In [ ]:
import pyelegant as pe
from pyelegant import elebuilder
from pyelegant import remote
from pyelegant import sdds
from pyelegant import util
from pyelegant import notation

In [ ]:
# Based on nsls2cb/20190315_Pelegant/cbiiMatch04.ele

In [ ]:
ed = elebuilder.EleDesigner('temp_hybridsimplex.ele', double_format='.12g')

ed.add_block('run_setup',
    lattice = 'lattice3.lte', p_central_mev = 3e3,
    use_beamline='ring',
    semaphore_file = '%s.done', parameters = '%s.param', default_order = 2
)

ed.add_newline()

In [ ]:
#ed.add_comment('! Load solution from previous optimization')
#ed.add_block('load_parameters',
#    filename = 'cbiiMatch02.param',
#    change_defined_values = 1,
#)

#ed.add_newline()

In [ ]:
ed.add_comment('! Ask for twiss parameters (during optimization)')
ed.add_block('twiss_output',
    output_at_each_step = 1,
    matched = 1,
    radiation_integrals = 1,
    concat_order = 2,
)

ed.add_newline()

ed.add_comment('! Ask for floor coordinates (during optimization)')
ed.add_block('floor_coordinates',
    filename = '%s.flr'
)

ed.add_newline()

ed.add_comment('! Load floor coordinate data for NSLS-II')
ed.add_comment('! - First ID')
ed.add_block('rpn_load',
    filename = 'nsls2.flr',
    tag = 'flr1',
    match_column = 'ElementName',
    match_column_value = 'MID',
    matching_row_number = 0
)
ed.add_comment('! - Second ID')
ed.add_block('rpn_load',
    filename = 'nsls2.flr',
    tag = 'flr2',
    match_column = 'ElementName',
    match_column_value = 'MID',
    matching_row_number = 1
)

ed.add_newline()

ed.add_block('run_control')

ed.add_newline()

In [ ]:
ed.add_comment('! Use parallel hybrid simplex optimization')
ed.add_block('parallel_optimization_setup',
    mode = 'minimize', method = 'hybridsimplex',
    hybrid_simplex_comparison_interval = 100,
    target = 1e-6, tolerance = 1e-14,
    #n_passes = 3, n_evaluations = 1500, n_restarts = 10,
    n_passes = 3, n_evaluations = 100, n_restarts = 1,
    verbose = 0, log_file = '/dev/tty',
    output_sparsing_factor = 100,
    term_log_file = '%s.tlog',
    population_log = '%s.pop',
    simplex_log = '%s.simlog',
    #!! Depending on your filesystem, you may want to
    #!! increase this in order to improve performance
    #simplex_log_interval = 10,
    simplex_log_interval = 50,
)

ed.add_newline()

In [ ]:
ed.add_comment('! Dipole angles')

if True:
    
    ed.add_block('optimization_variable',
        name = 'B1QDH', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    ed.add_block('optimization_variable',
        name = 'B2QDH', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    ed.add_block('optimization_variable',
        name = 'B3QDH', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    ed.add_block('optimization_variable',
        name = 'B4QDH', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    ed.add_block('optimization_variable',
        name = 'QF1', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    ed.add_block('optimization_variable',
        name = 'QF2', item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5
    )
    
else:
    
    for name in ['B1QDH', 'B2QDH', 'B3QDH', 'B4QDH', 'QF1', 'QF2']:
        ed.add_block('optimization_variable',
            name = name, item = 'ANGLE',
            lower_limit=-0.01, upper_limit=0.03, step_size=1e-5)
    
ed.add_newline()

In [ ]:
notation.convert_infix_to_rpn(
    'dtor(6) - B1QDH.ANGLE * 2 - B2QDH.ANGLE * 2 - B4QDH.ANGLE * 2 - QF1.ANGLE - QF2.ANGLE', 
    temp_repl=[('.ANGLE', '_DOT_ANGLE')])

In [ ]:
notation.convert_infix_to_rpn('selt(QF4.ANGLE, 0.01, 1e-6)', 
                              temp_repl=[('.ANGLE', '_DOT_ANGLE')])

In [ ]:
ed.add_comment('! Set QF4 angle so that the total per cell is 6 degrees')
ed.add_block('optimization_covariable',
    name = 'QF4', item = 'ANGLE',
    equation = '6 dtor B1QDH.ANGLE 2 * - B2QDH.ANGLE 2 * - B3QDH.ANGLE 2 * - B4QDH.ANGLE 2 * - QF1.ANGLE - QF2.ANGLE -'
)

ed.add_newline()

ed.add_comment('! Constrain QF4 angle within a reasonable range')
ed.add_block('optimization_term',
    term = "QF4.ANGLE 0.01 1e-6 selt"
)
ed.add_block('optimization_term',
    term = "QF4.ANGLE 0.03 1e-6 segt"
)

ed.add_newline()

In [ ]:
ed.add_comment('! Vary focusing gradients')

if True:
    ed.add_block('optimization_variable',
        name = 'B1QDH', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'B2QDH', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'B3QDH', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'B4QDH', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QF1', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QF2', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QF3', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QF4', item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4
    )
    
else:
    
    for name in ['B1QDH', 'B2QDH', 'B3QDH', 'B4QDH', 'QF1', 'QF2', 'QF3', 'QF4']:
        ed.add_block('optimization_variable',
            name = name, item = 'K1', lower_limit=-25, upper_limit=25, step_size=1e-4)
    
ed.add_newline()

In [ ]:
if True:
    ed.add_block('optimization_variable',
        name = 'M1G4A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'M2G4A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )

    ed.add_newline()

    ed.add_block('optimization_variable',
        name = 'Q0', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QH3G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QH2G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QH1G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )

    ed.add_newline()

    ed.add_block('optimization_variable',
        name = 'QL0', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QL3G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QL2G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    ed.add_block('optimization_variable',
        name = 'QL1G2A', item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4
    )
    
else:
    
    for name in ['M1G4A', 'M2G4A', 
                 'Q0', 'QH3G2A', 'QH2G2A', 'QH1G2A', 
                 'QL0', 'QL3G2A', 'QL2G2A', 'QL1G2A']:
        ed.add_block('optimization_variable',
            name = name, item = 'K1', lower_limit = -7, upper_limit = 7, step_size = 1e-4)

ed.add_newline()

In [ ]:
notation.convert_infix_to_rpn('abs(dnux/dp * 15) / 10.0', 
                              temp_repl=[('/dp', '_SLASH_dp')])

In [ ]:
notation.convert_infix_to_rpn('selt(Jx, 1, 1e-6)')

In [ ]:
notation.convert_infix_to_rpn('sene(MID#1.etax, 0, 1e-4)',
                              temp_repl=[('#', '_POUND_'), ('.etax', '_DOT_etax')])

In [ ]:
ed.add_block('optimization_term',
    term = "dnux/dp 15 * abs 10 /")
ed.add_block('optimization_term',
    term = "dnuy/dp 15 * abs 10 /"
)

ed.add_newline()

ed.add_comment('! Want Jx:[1, 2]')
ed.add_block('optimization_term',term = "Jx 1 1e-6 selt")
ed.add_block('optimization_term',term = "Jx 2 1e-6 segt")

ed.add_newline()

ed.add_comment('! Want etax to be zero in both ID straights')
ed.add_block('optimization_term',term = "MID#1.etax 0 1e-4 sene")
ed.add_block('optimization_term',term = "MID#2.etax 0 1e-4 sene")

ed.add_newline()

ed.add_comment('! Want etax>0.08 in high-dispersion region (weak requirement)')
ed.add_block('optimization_term',term = "MDISP#1.etax .1 .01 selt")
ed.add_block('optimization_term',term = "MDISP#2.etax .1 .01 selt")

ed.add_newline()

ed.add_comment('! Minimize the emittance')

ed.add_block('optimization_term',term = "ex0 1e12 * 50.0 1.0 segt 10 *")

ed.add_newline()

ed.add_comment('! Allow straight section lengths to vary (helps matching floor coordinates)')
ed.add_block('optimization_variable',
    name = 'ODL1G1A', item = 'L', lower_limit = 3.2, upper_limit = 3.4
)
ed.add_block('optimization_variable',
    name = 'ODH1G1A', item = 'L', lower_limit = 4.4, upper_limit = 4.6
)

ed.add_newline()

In [ ]:
notation.convert_infix_to_rpn(
    'sene(MID#1.betax / ODL1G1A.L, 1, .1) * 100',
    temp_repl=[('#', '_POUND_'), ('.beta', '_DOT_beta')])

In [ ]:
notation.convert_infix_to_rpn(
    'sene(sqrt( sqr(MID#1.X) + sqr(MID#1.Z) ), sqrt( sqr(flr1.X) + sqr(flr1.Z) ), 1e-3)',
    temp_repl=[('#', '_POUND_'), ('.X', '_DOT_X'), ('.Z', '_DOT_Z')])

In [ ]:
ed.add_comment('! Try to make betax=betay=L/2 in IDs, where L is the *total* straight length')
ed.add_comment('!&optimization_term term = "MID#1.betax ODL1G1A.L / 1 .1 sene 100 *" &end')
ed.add_comment('!&optimization_term term = "MID#1.betay ODL1G1A.L / 1 .1 sene 100 *" &end')
ed.add_comment('!&optimization_term term = "MID#2.betax ODH1G1A.L / 1 .1 sene 100 *" &end')
ed.add_comment('!&optimization_term term = "MID#2.betay ODH1G1A.L / 1 .1 sene 100 *" &end')
ed.add_comment('! Try to make betax & betay the same as NSLS-II Day-1 bare')
ed.add_block('optimization_term',term = "MID#1.betax 1.846 0.2 sene 5 *")
ed.add_block('optimization_term',term = "MID#1.betay 1.171 0.1 sene 5 *")
ed.add_block('optimization_term',term = "MID#2.betax 20.466 2.0 sene 5 *")
ed.add_block('optimization_term',term = "MID#2.betay 3.369 0.3 sene 5 *")

ed.add_newline()

ed.add_comment('! Keep the radius within 1mm of NSLS-II')
ed.add_block('optimization_term',
    term = "MID#1.X sqr MID#1.Z sqr + sqrt  flr1.X sqr flr1.Z sqr + sqrt 1e-3 sene")
ed.add_block('optimization_term',
    term = "MID#2.X sqr MID#2.Z sqr + sqrt  flr2.X sqr flr2.Z sqr + sqrt 1e-3 sene")

ed.add_newline()

ed.add_comment('! Make sure beta is not too large anywhere')
ed.add_block('optimization_term',term = "max.betax 35.0 1.0 segt")
ed.add_block('optimization_term',term = "max.betay 35.0 1.0 segt")

ed.add_newline()

ed.add_comment('! Make sure beta is not too small anywhere')
ed.add_block('optimization_term',term = "min.betax 0.2 0.1 selt")
ed.add_block('optimization_term',term = "min.betay 0.2 0.1 selt")

ed.add_newline()

ed.add_comment("! Ensure that central particle isn't lost")
ed.add_block('optimization_term',term = "Particles 1 1e-10 sene")

ed.add_newline()

ed.add_comment("! Beam consists of central particle only")
ed.add_block('bunched_beam')

ed.add_newline()

In [ ]:
ed.add_comment("! Start optimization")
ed.add_block('optimize')

ed.add_newline()

ed.add_comment("! Evaluate the results of optimization")

ed.add_newline()

ed.add_block('run_setup',
    lattice = 'lattice3.lte',
    use_beamline='ring',
    # ^ Note that here I am using the full ring, not 2 ring cells
    p_central_mev = 3e3,
    semaphore_file = '%s.done',
    magnets = '%s.mag',
    default_order = 2,
)

ed.add_newline()

ed.add_block('load_parameters',filename = '%s.param', change_defined_values = 1)

ed.add_newline()

ed.add_block('twiss_output',
    filename = '%s.twi',
    matched = 1,
    radiation_integrals = 1,
    concat_order = 2)

ed.add_newline()

ed.add_block('floor_coordinates', filename = '%s.flr')

ed.add_newline()

ed.add_block('save_lattice', filename = '%s.newlte')

In [ ]:
ed.write()

In [ ]:
output_filepath_list = ed.actual_output_filepath_list
output_filepath_list

In [ ]:
# Takes ~2.5 min if run

show_progress_plot = True
#show_progress_plot = False

username = getpass.getuser()

if not show_progress_plot: # If you don't care to see the progress of the optimization

    # Run Pelegant
    #
    # Minimal options
    #    remote_opts = dict(pelegant=True, ntasks=50)
    remote_opts = dict(
        use_sbatch=True, exit_right_after_sbatch=False, pelegant=True,
        #job_name='job', partition='debug', ntasks=18,
        job_name='job', partition='short', ntasks=18,
        time='7:00',
        #mail_type_begin=True,
        mail_type_end=True, mail_user=f'{username}@bnl.gov',
    )

    remote.run(remote_opts, ed.ele_filepath)
    # ^ This will block until the optimization is completed.

else: # If you want to see the progress of the optimization

    if not any([_v.endswith('.simlog') for _v in output_filepath_list]):
        raise RuntimeError('simplex_log is NOT specified. Cannot monitor simplex progress.')
    
    # Run Pelegant
    remote_opts = dict(
        use_sbatch=True, exit_right_after_sbatch=True, pelegant=True,
        #job_name='job', partition='debug', ntasks=50,
        job_name='job', partition='short', ntasks=18,
        time='7:00',
        #mail_type_begin=True,
        mail_type_end=True, mail_user=f'{username}@bnl.gov',
    )
    job_info = remote.run(remote_opts, ed.ele_filepath)

    # Start plotting simplex optimization progress
    remote.monitor_simplex_log_progress(job_info['job_ID_str'],
                                        ed.ele_filepath.replace('.ele', '.simlog'))

    try:
        os.remove(job_info['sbatch_sh_filepath'])
    except IOError:
        print('* Failed to delete temporary sbatch shell file "{}"'.format(
            job_info['sbatch_sh_filepath']))

In [ ]:
# Consolidate data in the generated SDDS files
print('Consolidating SDDS data files...')
sys.stdout.flush()
output, meta = {}, {}
for sdds_fp in output_filepath_list:
    if sdds_fp.startswith('/dev/'):
        continue
    print('Processing "{}"...'.format(sdds_fp))
    ext = sdds_fp.split('.')[-1]
    try:
        output[ext], meta[ext] = sdds.sdds2dicts(sdds_fp)
    except:
        continue
print('Finished.')
sys.stdout.flush()

In [ ]:
output

In [ ]:
meta

In [ ]:
# Save results into a HDF5 file
print('Writing data to HDF5 file...')
sys.stdout.flush()
output_filepath = 'hybrid_simplex_results.hdf5'
util.robust_sdds_hdf5_write(
    output_filepath, [output, meta], nMaxTry=10, sleep=10.0)
print('Finished.')
sys.stdout.flush()

In [ ]:
# Save the dictionaries into a gzipped pickle file
print('Writing data to pgz file...')
sys.stdout.flush()
output_filepath = 'hybrid_simplex_results.pgz'
mod_output = {}
for k, v in output.items():
    mod_output[k] = {}
    if 'params' in v:
        mod_output[k]['scalars'] = v['params']
    if 'columns' in v:
        mod_output[k]['arrays'] = v['columns']
mod_meta = {}
for k, v in meta.items():
    mod_meta[k] = {}
    if 'params' in v:
        mod_meta[k]['scalars'] = v['params']
    if 'columns' in v:
        mod_meta[k]['arrays'] = v['columns']
util.robust_pgz_file_write(
    output_filepath, [mod_output, mod_meta], nMaxTry=10, sleep=10.0)
print('Finished.')
sys.stdout.flush()

In [ ]:
# Delete the raw SDDS files
if True:
    for fp in output_filepath_list:
        if fp.startswith('/dev'):
            continue

        if fp.endswith('.simlog'):
            fp_list = glob.glob(fp + '-*')
            for sub_fp in fp_list:
                try:
                    os.remove(sub_fp)
                except:
                    print(f'Failed to delete "{sub_fp}"')

        elif fp.startswith('/dev'):
            continue

        else:
            try:
                os.remove(fp)
            except:
                print(f'Failed to delete "{fp}"')

In [ ]:
# Delete the temporary ELE file
if True:
    os.remove(ed.ele_filepath)